In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
from sklearn.metrics import average_precision_score, precision_recall_curve, accuracy_score
from options.test_options import TestOptions
from data import create_dataloader


def validate(model, opt):
    data_loader = create_dataloader(opt)

    with torch.no_grad():
        y_true, y_pred = [], []
        for img, label in data_loader:
            in_tens = img.cuda()
            y_pred.extend(model.forward(in_tens).sigmoid().flatten().tolist())
            y_true.extend(label.flatten().tolist())

    y_true, y_pred = np.array(y_true), np.array(y_pred)
    r_acc = accuracy_score(y_true[y_true==0], y_pred[y_true==0] > 0.5)
    f_acc = accuracy_score(y_true[y_true==1], y_pred[y_true==1] > 0.5)
    acc = accuracy_score(y_true, y_pred > 0.5)
    ap = average_precision_score(y_true, y_pred)
    return acc, ap, r_acc, f_acc, y_true, y_pred

In [8]:
import torch, gc
del model.model
gc.collect()
torch.cuda.empty_cache()

In [7]:
import sys
import time
import os
import csv
import torch
from options.test_options import TestOptions
import numpy as np
import random
import pandas as pd

from trainer import Trainer

DetectionTests = {
                'ForenSynths': { 'dataroot'   : '../../datasets/ForenSynths/',
                                 'no_resize'  : True,
                                 'no_crop'    : False,
                               },

           'GANGen-Detection': { 'dataroot'   : '../../datasets/GANGen-Detection/',
                                 'no_resize'  : True,
                                 'no_crop'    : False,
                               },

         'DiffusionForensics': { 'dataroot'   : '../../datasets/DiffusionForensics/',
                                 'no_resize'  : True,
                                 'no_crop'    : False,
                               },

        'UniversalFakeDetect': { 'dataroot'   : '../../datasets/UniversalFakeDetect/',
                                 'no_resize'  : True,
                                 'no_crop'    : False,
                               },

        'Diffusion1kStep': { 'dataroot'   : '../../datasets/Diffusion1kStep/',
                                 'no_resize'  : True,
                                 'no_crop'    : True,
                               },

                 }


opt = TestOptions()

# params
opt.model_path = './pretrained/xception-b5690688.pth'
opt.batch_size = 256
opt.isTrain = False
opt.mode = 'binary'
opt.class_bal = False
opt.eval = True
opt.num_threads = 8
opt.rz_interp = ['bilinear']
opt.loadSize = 256
opt.blur_prob = 0
opt.jpg_prob = 0
opt.cropSize = 299

print(f'Model_path {opt.model_path}')
torch.cuda.set_device('cuda:0')

# get model
mode = "Both"
model = Trainer([0], mode, opt.model_path)
# model.load(opt.model_path)
model.model.eval()
model.model.cuda()

results = []

for testSet in DetectionTests.keys():
    dataroot = DetectionTests[testSet]['dataroot']
    print(testSet)

    accs = []
    aps = []
    print(time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime()))
    for v_id, val in enumerate(os.listdir(dataroot)):
        opt.dataroot = '{}/{}'.format(dataroot, val)
        opt.classes  = '' #os.listdir(opt.dataroot) if multiclass[v_id] else ['']
        opt.no_resize = DetectionTests[testSet]['no_resize']
        opt.no_crop   = DetectionTests[testSet]['no_crop']
        acc, ap, _, _, _, _ = validate(model, opt)
        accs.append(acc)
        aps.append(ap)
        print("({} {:12}) acc: {:.1f}; ap: {:.1f}".format(v_id, val, acc*100, ap*100))
        results.append({'group': testSet, 'id': v_id, 'dataset': val, 'accuracy': acc * 100, 'average_precision': ap * 100})
    results.append({'group': testSet, 'id': v_id + 1, 'dataset': 'Mean', 'accuracy': np.array(accs).mean() * 100, 'average_precision': np.array(aps).mean() * 100})    #TODO: изменить dataframe
    print("({} {:10}) acc: {:.1f}; ap: {:.1f}".format(v_id+1,'Mean', np.array(accs).mean()*100, np.array(aps).mean()*100))
    print('*'*25) 

df = pd.DataFrame(results)
df.to_csv('F3Net_results.csv', index=False)


Model_path ./pretrained/xception-b5690688.pth


/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/deepfake_detection_models/F3Net/models.py:240: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(

ForenSynths
2024_07_29_12_02_00
(0 biggan      ) acc: 48.9; ap: 50.0
(1 crn         ) acc: 50.8; ap: 48.9
(2 cyclegan    ) acc: 52.3; ap: 51.3
(3 deepfake    ) acc: 50.4; ap: 53.2
(4 gaugan      ) acc: 50.5; ap: 50.0
(5 imle        ) acc: 48.3; ap: 43.3
(6 progan      ) acc: 50.2; ap: 50.7
(7 san         ) acc: 52.7; ap: 53.6
(8 seeingdark  ) acc: 54.2; ap: 53.2
(9 stargan     ) acc: 50.1; ap: 51.1
(10 stylegan    ) acc: 49.0; ap: 42.2
(11 stylegan2   ) acc: 49.2; ap: 45.3
(12 whichfaceisreal) acc: 48.4; ap: 46.2
(13 Mean      ) acc: 50.4; ap: 49.2
*************************
GANGen-Detection
2024_07_29_12_13_45
(0 AttGAN      ) acc: 52.5; ap: 53.3
(1 BEGAN       ) acc: 50.0; ap: 54.8
(2 MMDGAN      ) acc: 50.0; ap: 53.9
(3 RelGAN      ) acc: 55.1; ap: 56.5
(4 S3GAN       ) acc: 53.0; ap: 54.6
(5 SNGAN       ) acc: 50.0; ap: 51.6
(6 STGAN       ) acc: 50.0; ap: 59.4
(7 CramerGAN   ) acc: 50.0; ap: 52.3
(8 InfoMaxGAN  ) acc: 50.0; ap: 43.2
(9 Mean      ) acc: 51.2; ap: 53.3
**************

RuntimeError: Caught RuntimeError in DataLoader worker process 7.
Original Traceback (most recent call last):
  File "/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/env/almas-env/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 309, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/env/almas-env/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
  File "/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/env/almas-env/lib/python3.9/site-packages/torch/utils/data/_utils/collate.py", line 317, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/env/almas-env/lib/python3.9/site-packages/torch/utils/data/_utils/collate.py", line 174, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) f